# Introduction
---

# Import libries

In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load the data